## https://leetgpu.com/challenges/color-inversion#

In [3]:
import triton
import triton.language as tl

@triton.jit
def invert_kernel(
    image_ptr,
    width, height,
    BLOCK_SIZE: tl.constexpr
):
    image_ptr = image_ptr.to(tl.pointer_type(tl.uint8))
    p_id=tl.program_id(0)
    block_start=BLOCK_SIZE*p_id
    offsets=block_start+tl.arange(0,BLOCK_SIZE)
    mask=offsets< (4 * width * height)

    a=tl.load(image_ptr+offsets,mask=mask)

    idx=offsets%4
    val = tl.where(idx < 3, 255 - a, a)

    tl.store(image_ptr+offsets,val,mask=mask)


def solve(image_ptr: int, width: int, height: int):
    BLOCK_SIZE = 1024
    n_pixels = width * height * 4
    grid = (triton.cdiv(n_pixels, BLOCK_SIZE),)
    
    invert_kernel[grid](
        image_ptr,
        width, height,
        BLOCK_SIZE
    ) 